In [1]:
import matplotlib
#matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import sys
sys.path.append("/home/cat/code/widefield/") # go to parent dir

# 
import numpy as np
import os

# 
from tqdm import tqdm, trange

#  SVM module
from svm.svm import PredictSVMChoiceSuperSession


Autosaving every 180 seconds


In [2]:
def get_data_pie_charts(n_sec_lockout,
                        body_feats_lockout,
                        animal_id,
                        root_dir):
    
    # skip 1,2,3 n-sec-lockouts and the empty lockouts
    if len(body_feats_lockout)==0:    # if not using any lockout window
        if n_sec_lockout in [1,2, 3]:  # 
            return [], [], []


    # 
    fname = os.path.join(root_dir,
                         animal_id,
                         'super_sessions',
                         'alldata_body_and_nonreward_lockout_'+str(n_sec_lockout)+'secLockout_'+
                         str(body_feats_lockout)+'bodyfeats.npz')

    # 
    data = np.load(fname, allow_pickle=True)

    trials = data['trials']
    
    # 
    behaviors = data['behaviors']
    # 
    names = data['names']

    
    return trials, names, behaviors


# 
def get_no_of_hours(root_dir,
                    animal_id):
    
    
    sessions = os.listdir(os.path.join(root_dir,
                           animal_id,
                           'tif_files'))
    
    # 
    n_sessions=0
    n_rewarded=0
    for session in sessions:
        fname = os.path.join(root_dir,
                            animal_id,
                            'tif_files',
                            session,
                            'rewarded_times.txt')
        
        if os.path.exists(fname):
            n_sessions+=1

            temp = np.loadtxt(fname, dtype='float32', ndmin=1)
            if temp.shape[0]>0:
                n_rewarded+=temp.shape[0]

            
    return n_sessions, n_rewarded


In [3]:
######################################################################
######### GENERATE REWARDED (04) AND UNREWARDED (02) TIMES ###########
######################################################################

# this function is for making rewarded/unrewarded times and generating the final data+label .npz files
root_dir = '/media/cat/4TBSSD/yuki/'

#
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
#animal_ids = ['IA1']

#
n_sec_lockouts = [0,1,2,3,6,9,12,15,18,21,24,27,30]
#n_sec_lockouts = [2]

#
body_feats_lockouts = [[],[0],[1],[5],[0,1,5]]
body_feats_lockouts = [[],[0],[5]]
#body_feats_lockouts = [[]]

labels = ['all', 'left paw','right paw','licking',' left paw, right paw and licking']
labels = ['all', 'left paw', 'licking']

clrs = ['black','red','green']

#
fig=plt.figure()
ctr2 = 0
all_array = []
for k in range(len(animal_ids)):
    all_array.append([])
    #left_array.append([])
    #lick_array.append([])

    
for ctr_animal, animal_id in enumerate(animal_ids):

    # 
    #ax=plt.subplot(2,3,ctr2+1)
    ax=plt.subplot(1,1,1)
    
    # 
    ctr=0
    for body_feats_lockout in body_feats_lockouts:
        
        rewarded = []
        nSec = []
        for n_sec in tqdm(n_sec_lockouts):
            if len(body_feats_lockout)>0:
                if n_sec==0:
                    continue

            trials, names, _ = get_data_pie_charts(n_sec,
                                                    body_feats_lockout,
                                                    animal_id,
                                                    root_dir)
            # 
            n_rew = 0       # reset rewarded and nonrewarded plots
            n_nonrew = 0

            # 
            if len(trials)>0:
                for k in range(len(trials)):
                    n_rew += trials[k].shape[0]

                if n_sec==0:
                    n_sec=3

                
                rewarded.append(n_rew)
                nSec.append(n_sec)

                
        if nSec[0]==3:
            ntemp_rew = []
            ntemp_sec = []
            for k in range(3):
                ntemp_rew.append(rewarded[0])
                ntemp_sec.append(k)
                
            ntemp_sec.append(nSec)
            ntemp_rew.append(rewarded)

            nSec = np.hstack(ntemp_sec)
            rewarded = np.hstack(ntemp_rew)
            
        rewarded = np.hstack(rewarded)
        #rewarded = rewarded/1351
        plt.plot(nSec,
                 rewarded, 
                 c=clrs[ctr],
                 label=labels[ctr],
                linewidth=5)
        
        all_array[ctr_animal].append(rewarded)
        
        ctr+=1

    plt.legend()
    #plt.semilogy()    
    #plt.scatter(n_sec, n_nonrew)
    plt.xlim(0,20)
    plt.ylim(bottom=0.01)
    ctr2+=1
    
    if False:
        plt.savefig('/home/cat/decays.svg')
        plt.close()
        break
    else:
        
        pass
plt.show()


 85%|████████▍ | 11/13 [00:07<00:01,  1.54it/s]


KeyboardInterrupt: 

In [4]:
########################################################
######## PLOT # LEVER PULLS VS LOCKOUTS - FIG 3E #######
########################################################

fig=plt.figure()

print (all_array)

clrs = ['black','red','green']
#####################################
rew_all = []
for k in range(6):
    temp = all_array[k][0]
    #temp = temp/np.max(temp)
    rew_all.append(temp)

rew_all = np.vstack(rew_all)
rew_all = rew_all[:,1:]
x = np.array([1,2,3,6,9,12,15,18,21,24,27,30])

#
if False:
    mean = np.mean(rew_all, axis=0)
    std = np.std(rew_all, axis=0)    
    plt.plot(x,
             mean,
             color=clrs[0],
             linewidth=5)

    plt.fill_between(x, mean+std, mean-std, 
                     color=clrs[0], alpha=.1)
else:
    mean = np.mean(rew_all, axis=0)
    std = np.std(rew_all, axis=0)    
    plt.plot(x,
             mean,
             color=clrs[0],
             linewidth=5)
    for p in range(6):
        plt.scatter(x,rew_all[p],
                   c=clrs[0],
                    edgecolor='black',
                   alpha=.8)
    
# ######################################################
x = np.array([1,2,3,6,9,12,15,18,21,24,27,30])

rew_all = []
for k in range(6):
    temp = np.zeros(x.shape[0])
    temp2 = all_array[k][1]
    #temp2 = temp2/np.max(temp2)
    print (temp2.shape)
    temp[:temp2.shape[0]]=temp2
    
    rew_all.append(temp)

rew_all = np.vstack(rew_all)

#
if False:
    mean = np.mean(rew_all, axis=0)
    std = np.std(rew_all, axis=0)    
    plt.plot(x,
         mean,
             color=clrs[1],
         linewidth=5)

    plt.fill_between(x, mean+std, mean-std, 
                     color=clrs[1], alpha=.1)
else:
    mean = np.mean(rew_all, axis=0)
    std = np.std(rew_all, axis=0)    
    plt.plot(x,
         mean,
             color=clrs[1],
         linewidth=5)
    for p in range(6):
        plt.scatter(x,rew_all[p],
                   c=clrs[1],
                    edgecolor='black',
                   alpha=.8)
    
# ######################################################
x = np.array([1,2,3,6,9,12,15,18,21,24,27,30])

rew_all = []
for k in range(6):
    temp = np.zeros(x.shape[0])
    temp2 = all_array[k][2]
    #temp2 = temp2/np.max(temp2)
    print (temp2.shape)
    temp[:temp2.shape[0]]=temp2
    
    rew_all.append(temp)

rew_all = np.vstack(rew_all)

#
if False:
    mean = np.mean(rew_all, axis=0)
    std = np.std(rew_all, axis=0)    
    plt.plot(x,
         mean,
             color=clrs[2],
         linewidth=5)

    plt.fill_between(x, mean+std, mean-std, 
                     color=clrs[2], alpha=.1)
else:
    mean = np.mean(rew_all, axis=0)
    std = np.std(rew_all, axis=0)    
    plt.plot(x,
         mean,
             color=clrs[2],
         linewidth=5)
    
    for p in range(6):
        plt.scatter(x,rew_all[p],
                   c=clrs[2],
                    edgecolor='black',
                   alpha=.8)
    
#
plt.plot([3,3],[0,10000],'--',c='black')
plt.semilogy()
plt.fill_between(x, np.zeros(x.shape[0]), np.zeros(x.shape[0])+200,
                     color='black', alpha=.05)
plt.xlim(0,20)
#plt.ylim(bottom=1)

plt.legend()
plt.ylim(bottom=1)

plt.savefig('/home/cat/decays_all.svg')
plt.close()

        
        

[[array([1351, 1351, 1351, 1351, 1068,  741,  601,  503,  449,  399,  356,
        339,  311]), array([190, 150, 122,  62,  31,  19,  12,   8,   7,   7,   4,   3]), array([300, 261, 236, 124,  57,  40,  30,  25,  23,  19,  16,  13])], [], [], [], [], []]


IndexError: list index out of range

In [5]:
###################################################
##### VISUALIZE # OF HRS OF RECORDING - FIG 3A ####
###################################################
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
animal_names = ['M1','M2','M3','M4','M5','M6']

# 
fig=plt.figure()
ax1=plt.subplot(111)
ax2 = ax1.twinx()
for ctr, animal_id in enumerate(animal_ids):
    n_hrs, n_rew = get_no_of_hours(root_dir,animal_id)
    
    n_hrs = n_hrs*22/60.
    
    ax1.scatter(ctr, n_hrs, c='black',
                s=200,
                label=animal_id)
    
    ax2.scatter(ctr, n_rew, c='blue',
                s=200,
                label=animal_id)
    
    
# 
ax1.set_ylabel("# of hours of recording")
ax2.set_ylabel("# of rewarded pulls")
plt.xticks(np.arange(6), animal_names)
ax1.set_ylim(0,45)
ax2.set_ylim(0,7500)

# 
if True:
    plt.savefig('/home/cat/n_hours.svg')
    plt.close()
else:
    plt.show()

<ipython-input-2-a1a2827547ed>:55: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Apr22_Week2_30Hz/rewarded_times.txt"
  temp = np.loadtxt(fname, dtype='float32', ndmin=1)
<ipython-input-2-a1a2827547ed>:55: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/IA1/tif_files/IA1am_May2_Week4_30Hz/rewarded_times.txt"
  temp = np.loadtxt(fname, dtype='float32', ndmin=1)
<ipython-input-2-a1a2827547ed>:55: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb15_30Hz/rewarded_times.txt"
  temp = np.loadtxt(fname, dtype='float32', ndmin=1)
<ipython-input-2-a1a2827547ed>:55: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/IA2/tif_files/IA2am_Mar10_30Hz/rewarded_times.txt"
  temp = np.loadtxt(fname, dtype='float32', ndmin=1)
<ipython-input-2-a1a2827547ed>:55: UserWarning: loadtxt: Empty input file: "/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec9_30Hz/rewarded_times.txt"
  temp = np.loadtxt(fname, d

In [322]:
# ################################################
# ##### VISUALIZE PIE CHARTS - SINGLE ANIMAL ####
# ################################################

# fname = '/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb9_30Hz/IA1pm_Feb9_30Hz_body_and_nonreward_lockout_0secLockout_[]bodyfeats.npz'
# d = np.load(fname, allow_pickle=True)
# trials = d['trials']
# random = d['random']
# behaviors = d['behaviors']
# names = d['names']

# print (behaviors.shape)
# print (names)
# print (behaviors[0], behaviors[3])

# selected_behaviors = [0,1,2,3,7]

# sizes = []
# for k in selected_behaviors:
#     sizes.append(behaviors[k].shape[0])

# # Pie chart, where the slices will be ordered and plotted counter-clockwise:
# labels = names[selected_behaviors]
# labels[-1]='licking'

# # 
# fig1, ax1 = plt.subplots()
# patches, texts,_ = ax1.pie(sizes, 
#                         #explode=explode, 
#                         #labels=labels, 
#                         autopct='%1.1f%%',
#                         #shadow=True, 
#                         pctdistance=1.1,
#                         labeldistance=1.2,
#                         startangle=180
#                        )
# plt.legend(patches, labels, loc="best")

# # Set aspect ratio to be equal so that pie is drawn as a circle.
# plt.axis('equal')
# plt.tight_layout()

# # 
# plt.show()



In [246]:
#########################################################
##### VISUALIZE PIE CHARTS - ALL SESSIONS - FIG 3b ######
#########################################################
#
animal_id = 'AQ2'    # 
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']

all_rewarded = []
plotting=False
for animal_id in animal_ids:

    # 
    n_sec = 0

    #
    body_feats_lockout = []

    ctr=0

    # 
    trials, names, behaviors = get_data_pie_charts(n_sec,
                                                   body_feats_lockout,
                                                   animal_id,
                                                   root_dir)

    # 
    sess = 3
    feat = 1
    print (np.vstack(trials).shape)
    print ("# sessions: ", len(behaviors))
    print ("seesion ",sess, " # of behav ", len(behaviors[sess]))
    print ("# of feature moves: ", behaviors[sess][feat].shape)


    # get # rewarded pulls:
    n_rew = 0
    n_nonrew = 0
    n_left_paw = 0
    n_right_paw = 0
    n_licking = 0

    # 
    feat_rew = 0
    feat_nonrew = 1
    feat_left_paw = 2
    feat_right_paw = 3
    feat_licking = 7

    n_sess = 0
    min_quiescence = 30 # in frame times
    for sess in trange(len(behaviors)):
        try:
            if len(behaviors[sess][feat_left_paw])>0:
                n_rew+= behaviors[sess][feat_rew].shape[0]
                n_nonrew+= behaviors[sess][feat_nonrew].shape[0]

                # left paw
                temp = behaviors[sess][feat_left_paw]
                diff = temp[1:]-temp[:-1]
                #print (diff)
                idx = np.where(diff>=min_quiescence)[0]
                n_left_paw+= idx.shape[0]

                # left paw
                temp = behaviors[sess][feat_right_paw]
                diff = temp[1:]-temp[:-1]
                idx = np.where(diff>=min_quiescence)[0]
                n_right_paw+= idx.shape[0]

                # left paw
                temp = behaviors[sess][feat_licking]
                diff = temp[1:]-temp[:-1]
                idx = np.where(diff>=min_quiescence)[0]
                n_licking+= idx.shape[0]
            n_sess+=1

        except:
            pass

    print ("video sessions: ", n_sess)
    print ("nrew: ", n_rew)  
    print ("n_nonrew: ", n_nonrew)  
    print ("left_paw: ", n_left_paw)  
    print ("n_right_paw: ", n_right_paw)  
    print ("n_licking: ", n_licking)  


    sizes = np.array([n_rew, n_nonrew, n_left_paw, n_right_paw, n_licking])
    sizes = sizes/np.sum(sizes)*100

    # Pie chart, where the slices will be ordered and plotted counter-clockwise:
    labels = ['rewarded lever pulls','non-rewarded lever pulls','left paw','right paw', 'licking']

    # 
    all_rewarded.append(sizes[0])
    
    # 
    if plotting:

        fig1, ax1 = plt.subplots()
        colors = ['blue','lightgrey','wheat','paleturquoise','pink']
        patches, texts,_ = ax1.pie(sizes, 
                                #explode=explode, 
                                #labels=labels, 
                                colors=colors,
                                autopct='%1.1f%%',
                                #shadow=True, 
                                pctdistance=1.1,
                                labeldistance=1.2,
                                startangle=180,
                                textprops={'fontsize': 20}
                               )

        #patches[0][0].set_alpha(.1)

        labels = [f'{l}, {s:0.1f}%' for l, s in zip(labels, sizes)]

        ax1.legend(patches, labels, loc="best")

        # Set aspect ratio to be equal so that pie is drawn as a circle.
        ax1.axis('equal')
        plt.tight_layout()
        plt.title(animal_id)
        # 

        if True:
            plt.savefig('/home/cat/piechart_'+animal_id+'.svg')
            plt.close()
            break
        else:
            plt.show()
            break

   

100%|██████████| 53/53 [00:00<00:00, 23387.49it/s]

(1351, 1800, 16)
# sessions:  53
seesion  3  # of behav  9
# of feature moves:  (35,)
video sessions:  53
nrew:  619
n_nonrew:  2318
left_paw:  6438
n_right_paw:  6703
n_licking:  4471



100%|██████████| 33/33 [00:00<00:00, 39388.74it/s]

(1704, 1800, 16)
# sessions:  33
seesion  3  # of behav  8
# of feature moves:  (9,)
video sessions:  33
nrew:  294
n_nonrew:  537
left_paw:  1567
n_right_paw:  1746
n_licking:  0



100%|██████████| 41/41 [00:00<00:00, 22292.77it/s]

...

(1615, 1800, 16)
# sessions:  40
seesion  3  # of behav  8
# of feature moves:  (165,)
video sessions:  40
nrew:  376
n_nonrew:  1159
left_paw:  2225
n_right_paw:  2311
n_licking:  1187



100%|██████████| 39/39 [00:00<00:00, 19165.54it/s]

(1867, 1800, 16)
# sessions:  39
seesion  3  # of behav  8
# of feature moves:  (366,)
video sessions:  39
nrew:  354
n_nonrew:  2230
left_paw:  2622
n_right_paw:  2439
n_licking:  1225



100%|██████████| 79/79 [00:00<00:00, 21396.75it/s]

(6739, 1800, 16)
# sessions:  79
seesion  3  # of behav  8
# of feature moves:  (82,)
video sessions:  78
nrew:  2726
n_nonrew:  6982
left_paw:  10746
n_right_paw:  11327
n_licking:  9120
[3.012312034648888, 7.094594594594595, 5.259460002007428, 5.180490493248828, 3.9909808342728295, 6.664873719468961]


In [251]:
#####################################################
##### VISUALIZE % REWARDED LEVER PULLS - FIG 3c #####
#####################################################
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
animal_names = ['M1','M2','M3','M4','M5','M6']

# 
fig=plt.figure()
ax1=plt.subplot(111)
for ctr, animal_id in enumerate(animal_ids):
    n_hrs = all_rewarded[ctr]
    ax1.scatter(ctr, n_hrs, c='blue',
                s=200,
                label=animal_id)
       
# 
plt.plot([0,6],[np.mean(all_rewarded),np.mean(all_rewarded)],'--')
ax1.set_ylabel("% behaviors == lever pulls")
plt.xticks(np.arange(6), animal_names)
ax1.set_ylim(0,10)
#ax2.set_ylim(0,7500)

# 
if True:
    plt.savefig('/home/cat/percent_leverpulls.svg')
    plt.close()
else:
    plt.show()
    
    
print (all_rewarded)

print (np.mean(all_rewarded))

[3.012312034648888, 7.094594594594595, 5.259460002007428, 5.180490493248828, 3.9909808342728295, 6.664873719468961]
5.200451946373588



# Next - do the SVM decoding plots ###


In [3]:
##########################################
##########################################
##########################################
import scipy
from statsmodels.stats.multitest import multipletests

# 
def filter_trace(trace, smooth_window=30):

    box = np.ones(smooth_window)/smooth_window
    trace_smooth = np.convolve(trace, box, mode='valid')

    return trace_smooth


# 
def compute_significance(data):

    #
    significance = 0.05

    #
    sig = []
    for k in range(data.shape[0]):

        #
        res = scipy.stats.ttest_1samp(data[k], 0.5)

        sig.append(res[1])


    sig_save = np.array(sig).copy()

    # multiple hypothesis test Benjamini-Hockberg
    temp = np.array(sig)
    temp2 = multipletests(temp, 
                          alpha=significance, 
                          method='fdr_bh')
    sig = temp2[1]

    # find earliest significant;
    edt = 0
    for k in range(sig.shape[0]-1,0,-1):
        if sig[k]<=significance:
            edt = k+1
        else:
            break

    # 
    edtsec = -(sig.shape[0]-edt)/30.

    # 
    return edtsec

                  
        
#################################################
root_dir = '/media/cat/4TBSSD/yuki'

# lockouts = [0,3,6,9,12,15,18,21,24,27,30]
lockouts = [0,1,2,3,6,9,12,15,18,21,24,27,30]
bodyfeats = []


#body_feats_lockouts = [[],[0],[1],[5],[0,1,5]]
#body_feats_lockouts = [[],[0],[5]]
bodyfeats = [5]


# from matplotlib import 
cmap = plt.cm.get_cmap('viridis', len(lockouts))    # 11 discrete colors

#
fig = plt.figure()
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']

for animal_id in animal_ids:
    edts = []
    for ctr, lockout in enumerate(lockouts):

        edts_local=[]
        for n in trange(0, 10000, 50):

            fname = os.path.join(root_dir,
                                 animal_id,
                                 'super_sessions',
                                 'alldata_body_and_nonreward_lockout_'+str(lockout)+'secLockout_'+
                                 str(bodyfeats)+'bodyfeats_'+
                                 str(n)+'_super_res.npz')

            try:
                data = np.load(fname,
                          allow_pickle=True)
            except:
                print ("Fname not found: ", animal_id, lockout, bodyfeats)
                break

            acc = data['accuracy']

            # 
            acc2 = []
            for k in range(acc.shape[1]):
                acc2.append(filter_trace(acc[:,k]))

            acc2 = np.vstack(acc2).T
            #print (acc2.shape)
            if False:
                t = np.arange(acc2.shape[0])/30-29
                plt.plot(t,acc2.mean(1), c='black',alpha=.1)

                # 
                std = np.std(acc2,axis=1)
                plt.fill_between(t, acc2.mean(1)-std, acc2.mean(1)+std,
                                     color='black', alpha=.05)   


            edt = compute_significance(acc2)
            #print (edt)
            if False:
                plt.scatter(edt,0.5, 
                        c=cmap(ctr))
            edts_local.append(edt)

            #break
        if len(edts_local)>0:
            edts.append([lockout,edts_local])

    np.save('/home/cat/'+animal_id+"_edts_lockedout_licking.npy", edts)
#box_plots(edts_local)
#plt.xlim(-30,1)
#plt.ylim(0.4,1.0)
#plt.plot([-30,1],[0.5,0.5],'--',c='grey')
plt.show()

  0%|          | 0/200 [00:00<?, ?it/s]

Fname not found:  IA1 0 [5]


  0%|          | 0/200 [00:00<?, ?it/s]

Fname not found:  IA1 1 [5]


  0%|          | 0/200 [00:00<?, ?it/s]

...

  0%|          | 0/200 [00:00<?, ?it/s]

Fname not found:  IA2 1 [5]


  0%|          | 0/200 [00:00<?, ?it/s]


Fname not found:  IA2 2 [5]


KeyboardInterrupt: 

In [70]:
import pandas as pd
from scipy import stats
import matplotlib.patches as mpatches

    
# 
def box_plots(edts,
             licking_flag=False):
    
    pvals = [0.05,0.01,0.001,0.0001,0.00001]

    clrs_local = ['magenta','brown','pink','lightblue','darkblue', 'blue']
    
    # 
    if licking_flag==False:
        my_dict = dict(three = edts[0][1], 
                   six = edts[1][1],
                   nine = edts[2][1],
                   twelve = edts[3][1],
                   fifteen = edts[4][1],                      
                  #All = edts[5]+15,                      
                 )
    else:
        my_dict = dict(one = edts[0][1], 
               two = edts[1][1],
               three = edts[2][1],
               six = edts[3][1],
               #nine = edts[4][1],
               #twelve = edts[3][1],
               #fifteen = edts[4][1],                      
              #All = edts[5]+15,                      
             )
        
    cmap = plt.cm.get_cmap('Reds', len(my_dict))    # 11 discrete colors

        
    #
    data = pd.DataFrame.from_dict(my_dict, orient='index')
    data = data.transpose()

    print ("DATA: ", data.shape)
    print ("means: ", np.mean(data,0))
    means =np.mean(data,0)
    ######################################################### 
    ################## SCATTER PLOTS ########################
    ######################################################### 
    for i,d in enumerate(data):
        y = data[d]
        x = np.random.normal(i+1, 0.04, len(y))
        #print (x)
        #print (y)
        if licking_flag==False:
            plt.scatter(x,y,
                   c=clrs_local[i],
                   #c=cmap(i),
                    s=100,
                   edgecolor='black')
        else:
            plt.scatter(x,y,
                   #c=clrs_local[i],
                   c=cmap(i),
                    s=100,
                   edgecolor='black')

    ######################################################### 
    ###################### BOX PLOTS ########################
    ######################################################### 
    flierprops = dict(#marker='o', 
                      #markerfacecolor='g', 
                      markersize=10000,
                      linestyle='none', 
                      markeredgecolor='r')

    #
    data.boxplot(showfliers=False,
                 flierprops=flierprops)


    
    #
    plt.xlim(0.5, 6.5)
    plt.ylim(-15,0)    
    plt.xticks([])
    plt.yticks([])

    #
    plt.plot([0,6.5], [-3,-3],'--',linewidth=3,c='grey',alpha=.5)
    plt.plot([0,6.5], [-5,-5],'--',linewidth=3,c='grey',alpha=.5)
    plt.plot([0,6.5],[-10,-10],'--',linewidth=3,c='grey',alpha=.5)

    # ax2 = ax.twinx()  # instantiate a second axes that shares the same x-axis

    patches = []
    for p in range(1,len(my_dict),1):
        res = stats.ks_2samp(edts[p][1], edts[0][1])
        print ("res: ", res)
        label_ = ''
        for k in range(len(pvals)):
            if res[1]<pvals[k]:
                label_ = label_ + "*"
            else:
                break

        patches.append(mpatches.Patch(label=label_))

    plt.legend(handles=patches,fontsize=12)

    return means
    
############################
animal_ids = ['IA1','IA2','IA3','IJ1','IJ2','AQ2']
#animal_ids = ['IA1']#,'IA2','IA3','IJ1','IJ2','AQ2']
licking_flag = False
means = []
for animal_id in animal_ids:
    if licking_flag==False:
        edts = np.load(os.path.join('/home/cat/',
                                animal_id +
                                #'_edts_lockedout_licking.npy'), allow_pickle=True)
                                '_edts_lockedout.npy'), allow_pickle=True)
    else:
        edts = np.load(os.path.join('/home/cat/',
                                animal_id +
                                '_edts_lockedout_licking.npy'), allow_pickle=True)
                                #'_edts_lockedout.npy'), allow_pickle=True)

        
    #
    fig = plt.figure(figsize=(4.5,5.5))
    plt.title(animal_id)
    m_ = box_plots(edts, licking_flag)
    means.append(m_)
    if True:
        plt.savefig(os.path.join('/home/cat/',
                                animal_id+"_edts_lockdout_licking.svg"))
        plt.close()
    else:
        plt.show()

DATA:  (27, 5)
means:  three     -2.855556
six       -2.685714
nine      -1.991111
twelve    -2.430556
fifteen   -1.926667
dtype: float64
res:  KstestResult(statistic=0.164021164021164, pvalue=0.8517362612151119)
res:  KstestResult(statistic=0.3851851851851852, pvalue=0.08664813231587754)
res:  KstestResult(statistic=0.21296296296296297, pvalue=0.7798023139544679)
res:  KstestResult(statistic=0.3925925925925926, pvalue=0.15668362096582977)
DATA:  (34, 5)
means:  three     -4.010784
six       -4.790805
nine      -5.105000
twelve    -4.826190
fifteen   -3.139394
dtype: float64
res:  KstestResult(statistic=0.3022312373225152, pvalue=0.0890159924437578)
res:  KstestResult(statistic=0.14411764705882352, pvalue=0.9165595995181188)
res:  KstestResult(statistic=0.2815126050420168, pvalue=0.3435155245384486)
res:  KstestResult(statistic=0.35294117647058826, pvalue=0.18153108549860508)
DATA:  (49, 5)
means:  three     -5.181633
six       -6.209333
nine      -3.530303
twelve    -3.672222
fifteen 

In [76]:

fig=plt.figure()
t = np.array([3,6,9,12,15])
for k in range(6):
    mm = means[k]#.to_numpy()
    print (mm)
    plt.plot(t, mm, 
             label=animal_ids[k],
             linewidth=5)
    plt.scatter(t, mm,
               s=100)
    
    
plt.plot([3,15],[-3,-3],'--',c='grey')
plt.plot([3,15],[-5,-5],'--',c='grey')

plt.ylim(-6.5,0)
plt.legend()
if True:
    plt.savefig('/home/cat/mean_fits.svg')
    plt.close()
else:
    plt.show()

three     -2.855556
six       -2.685714
nine      -1.991111
twelve    -2.430556
fifteen   -1.926667
dtype: float64
three     -4.010784
six       -4.790805
nine      -5.105000
twelve    -4.826190
fifteen   -3.139394
dtype: float64
three     -5.181633
six       -6.209333
nine      -3.530303
twelve    -3.672222
fifteen   -2.266667
dtype: float64
three     -3.826042
six       -3.043860
nine      -2.356667
twelve    -2.747619
fifteen   -1.873333
dtype: float64
three     -4.171171
six       -2.950000
nine      -2.172222
twelve    -2.325000
fifteen   -1.644444
dtype: float64
three     -4.782716
six       -4.002825
nine      -2.884524
twelve    -2.729630
fifteen   -2.491667
dtype: float64


In [474]:
data = np.load('/media/cat/4TBSSD/yuki/IA2/super_sessions/alldata_body_and_nonreward_lockout_0secLockout_[5]bodyfeats.npz',
              allow_pickle=True)

names = data['names']
print (names)

[['rewarded_lever_pulls' 'unrewarded_lever_pulls' 'left_paw' 'right_paw'
  'nose' 'jaw' 'right_ear' 'tongue' 'lever_DLC']
 ['rewarded_lever_pulls' 'unrewarded_lever_pulls' 'left_paw' 'right_paw'
  'nose' 'jaw' 'right_ear' 'tongue' 'lever_DLC']
 ['rewarded_lever_pulls' 'unrewarded_lever_pulls' 'left_paw' 'right_paw'
  'nose' 'jaw' 'right_ear' 'tongue' 'lever_DLC']
 ['rewarded_lever_pulls' 'unrewarded_lever_pulls' 'left_paw' 'right_paw'
  'nose' 'jaw' 'right_ear' 'tongue' 'lever_DLC']
 ['rewarded_lever_pulls' 'unrewarded_lever_pulls' 'left_paw' 'right_paw'
  'nose' 'jaw' 'right_ear' 'tongue' 'lever_DLC']
 ['rewarded_lever_pulls' 'unrewarded_lever_pulls' 'left_paw' 'right_paw'
  'nose' 'jaw' 'right_ear' 'tongue' 'lever_DLC']]


In [480]:
data = np.load('/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Feb5_30Hz/IA2pm_Feb5_30Hz_0secNoMove_movements.npz',
              allow_pickle=True)

labels = data['labels']
print (labels.shape)
print (labels)

inits = data['feature_initiations']
print (inits.shape)
print (inits[5])
plt.plot(inits[5])
plt.show()

(7,)
['left_paw' 'right_paw' 'nose' 'jaw' 'right_ear' 'tongue' 'lever']
(7, 19986)
[0. 0. 0. ... 0. 0. 0.]
